<a href="https://colab.research.google.com/github/abhi147p/Abhishek_INFO5731_Spring2023/blob/main/In_class_exercise_02_02072023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The second In-class-exercise (02/07/2023, 40 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points): Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:





'''

Question 2 (10 points): Write python code to collect 1000 data samples you discussed above.

In [ ]:
# You code here (Please add comments in the code):



Question 3 (10 points): Write python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "information retrieval". The articles should be published in the last 10 years (2012-2022).

The following information of the article needs to be collected:

(1) Title

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [1]:
# You code here (Please add comments in the code):

import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd


def get_soup(url):
    """
        params1: url (contains the url of google scholar page)
        return: soup (fetching the url page data and then further converted to html parser)
    """
  # headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    try:
        data = requests.get(url, headers)
        # print(f"Extracted the data with response as {data.status_code}")
        if data.status_code != 200:
            raise Exception("Failed to fetch data")
    except Exception as ex:
        print(f"Exception occurred as {data.text} with status_code {data.status_code}")
        return None
    soup = BeautifulSoup(data.content)
    return soup

def get_title(title):
    return str(title.find("a").text)

def get_abstract_url(title):
    return str(title.find("a").get("href"))


def get_article_info(article):
    year = int(re.search(r'\d+', article.text).group())
    article = str(article.text).replace("\xa0", "")
    article = article.split("-")
    published = article[-1].strip()
    author = article[0].strip()
    return author, year, published

def get_tags(soup):
    all_titles = soup.findAll("h3", attrs={"class": "gs_rt"})
    all_authors = soup.findAll("div", attrs={"class": "gs_a"})
    all_abstracts = soup.findAll("div", attrs={"class": "gs_rs"})

    titles, authors, year, published, abstract, abs_url = [], [], [], [], [], []

    for title in all_titles:
        titles.append(get_title(title))
        abs_url.append(get_abstract_url(title))
 
    for author in all_authors:
        auth, yr, publs = get_article_info(author)
        authors.append(auth)
        year.append(yr)
        published.append(publs)
 
    for abstr in all_abstracts:
        abstract.append(get_abstract(abstr))
 
    return titles, authors, year, published, abstract, abs_url
 

def get_abstract(abstr):
    return str(abstr.text).replace("\n", "")


def fetch_web_data(records):
    year_st, year_end = 2012, 2022
    columns_google = ["Title", "Author", "Year", "Published", "Abstract", "Abstract_UrL"]
    # fetching for 1000 articles
    titles, authors, years, published, abstract, abs_url = [], [], [], [], [], []
    final_data = []
    print("***** BEFORE FETCHING *********")
    # records = 100 # no of articles
    for i in range(0, records, 10):
        url = f"https://scholar.google.com/scholar?start={i}&q=information+retrieval&hl=en&as_sdt=0,44&as_ylo={year_st}&as_yhi={year_end}&as_vis=1"
        soup = get_soup(url)
        # titles, authors, year, published, abstract, abs_url
        a, b, c, d, e, f = get_tags(soup)
        titles.extend(a)
        authors.extend(b)
        years.extend(c)
        published.extend(d)
        abstract.extend(e)
        abs_url.extend(f)
        print(f"******* fetched {(i+10)} articles *********")
        time.sleep(5)
   
    for i in range(100):
        final_data.append([titles[i],authors[i],years[i],published[i],abstract[i],abs_url[i]])

    print("******* AFTER FETCHING ********")
    df = pd.DataFrame(final_data, columns = columns_google)
    print(f"Number of records: {df.shape}")
    return df

# url = "https://scholar.google.com/scholar?start=200&q=information+retrieval&hl=en&as_sdt=0,44&as_ylo=2012&as_yhi=2022&as_vis=1"
# soup = get_soup(url)
# all_titles = soup.findAll("h3", attrs={"class": "gs_rt"})
# all_authors = soup.findAll("div", attrs={"class": "gs_a"})
# all_abstracts = soup.findAll("div", attrs={"class": "gs_rs"})

df = fetch_web_data(int(input("Enter num of articles: ")))
print(f"dimensions of articles: {df.shape}")
df.head()

Enter num of articles: 100
***** BEFORE FETCHING *********
******* fetched 10 articles *********
******* fetched 20 articles *********
******* fetched 30 articles *********
******* fetched 40 articles *********
******* fetched 50 articles *********
******* fetched 60 articles *********
******* fetched 70 articles *********
******* fetched 80 articles *********
******* fetched 90 articles *********
******* fetched 100 articles *********
******* AFTER FETCHING ********
Number of records: (100, 6)
dimensions of articles: (100, 6)


,Title,Author,Year,Published,Abstract,Abstract_UrL
0,Information retrieval as statistical translation,"A Berger, J Lafferty",2017,dl.acm.org,… There is a large literature on probabilistic...,https://dl.acm.org/doi/abs/10.1145/3130348.313...
1,A survey of automatic query expansion in infor...,"C Carpineto, G Romano",2012,dl.acm.org,… information retrieval systems is largely cau...,https://dl.acm.org/doi/abs/10.1145/2071389.207...
2,A language modeling approach to information re...,"JM Ponte, WB Croft",2017,dl.acm.org,"… models, we have developed an approach to ret...",https://dl.acm.org/doi/pdf/10.1145/3130348.313...
3,A study of smoothing methods for language mode...,"C Zhai, J Lafferty",2017,dl.acm.org,… to information retrieval are attractive and ...,https://dl.acm.org/doi/abs/10.1145/3130348.313...
4,Open language learning for information extraction,"M Schmitz, S Soderland, R Bart…",2012,aclanthology.org,Open Information Extraction (IE) systems extra...,https://aclanthology.org/D12-1048.pdf


Question 4 (10 points): Write python code to collect 1000 posts from Twitter, or Facebook, or Instagram. You can either use hashtags, keywords, user_name, user_id, or other information to collect the data. 

The following information needs to be collected:

(1) User_name

(2) Posted time

(3) Text 

In [3]:
# You code here (Please add comments in the code):

# hashtag

import tweepy
import pandas as pd

consumer_key = "u7L1lnR7HN85dn1qnTFO1cegb"
consumer_secret = "QN1JrEmit2To46ZcwWAT4aI5QGWZXWRDDUPnMCWV5M66SFc8wT"
access_key = "1144377060036620294-BSEicX3zH7hIhksbNZV9mrWFwa07cO"
access_secret = "gxWMOodDq1nQAjix9mHEOUSAtgE7XH5ctHInm0XRslJce"

def authenticate():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    return api

def fetch_tweets(api, num_of_tweets, hashtag):
    all_tweets = []
    for tweet in tweepy.Cursor(api.search, q='#' + hashtag).items(num_of_tweets):
        all_tweets.append(tweet)
    return all_tweets

def fetch_tweet_info(tweet):
    return [str(tweet.user.screen_name), str(tweet.text), str(tweet.created_at)]

tweet_api = authenticate()
num_of_tweets = int(input("Enter number of tweets want to fetch: "))
hashtag = input("Enter the hashtag: ")
all_tweets = fetch_tweets(tweet_api, num_of_tweets, hashtag)
tweet_info = list(map(fetch_tweet_info, all_tweets))
df = pd.DataFrame(tweet_info, columns=["username", "text", "posted_time"])
print(f"dimensions of tweets: {df.shape}")
df.head()

Enter number of tweets want to fetch: 1000
Enter the hashtag: Sports
dimensions of tweets: (1000, 3)


,username,text,posted_time
0,NewBertrand,1/Become Responsible\n#Artist #Culture #Sports...,2023-02-12 10:54:38
1,oldnsold2013,RT @rusticrelics04: https://t.co/pcJbAyVuMZ #c...,2023-02-12 10:54:37
2,getthatrightgtr,Leather Vs. Textile MC Riding Jackets https://...,2023-02-12 10:54:13
3,lovekntol,MASIH DGN TOPIK IBU2 PENJAGA WARUNG\n\nhttps:/...,2023-02-12 10:54:02
4,Tv4Winston,RT @nituchandra: This is great! #sports 👏👏👏😊 T...,2023-02-12 10:53:53
